In [186]:
!cd /content/
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
# https://drive.google.com/file/d/1KWnX3eMPJrzhsegi0LmyGUbUV5pqKw_R/view?usp=sharing
file_id = '1KWnX3eMPJrzhsegi0LmyGUbUV5pqKw_R'
downloaded = drive.CreateFile({'id':file_id})
downloaded.FetchMetadata(fetch_all=True)
downloaded.GetContentFile(downloaded.metadata['title'])
!unzip ss_dataset.zip -d .

Archive:  ss_dataset.zip
replace ./ss_dataset/3/210.bmp? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

## Segmentation dataset prepration

In [2]:
import numpy as np
import pandas as pd
import shutil
import tensorflow as tf
from zipfile import ZipFile
import keras.backend as K
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import os
from PIL import Image

We want to store our dataset in a pandas dataframe for fast and accurate process. Then we will convert these dataframe to a Tensor. so first let's define our helper functions

In [136]:
def dataframe_creation(image_path, name):
    """
    This function is storing path of the images in a dataframe beside of each image id (in image name).
    Walk though the image_path and read the dirpathes and images name in each dir.
    Then append each image full path in a list.
    Extract the image name without the full path and extension and append it to the ids list.
    Please make sure each full path in first list is correspond to the id in second list at the same index.
    Arguments:
      image_path: root directory full path
      name: name for column of full pathes in dataframe
    return:
      df: a df contains ids and full path of each image id. call the ids column, id and pathes column, name.
          Please set the ids column to be index in this df.

    """
    ##################################################
    ############### YOUR CODES GO HERE ###############
    image_paths = []
    ids = []

    for dir_path, _, file_names in os.walk(image_path):
      for file_name in file_names:
        image_paths.append(os.path.join(dir_path, file_name))
        _id = os.path.splitext(file_name)[0]
        if 'label' in _id:
          _id = _id.replace('_label', '')
        ids.append(_id)

    df = pd.DataFrame({'id':ids, name:image_paths})

    return df
    ##################################################

In [51]:
def display(display_list):
    plt.figure(figsize=(15, 15))

    title = ['Input Image', 'True Mask', 'Predicted Mask']

    for i in range(len(display_list)):
        plt.subplot(1, len(display_list), i+1)
        plt.title(title[i])
        # a function for showing images
        # please write a code to convert images from tensor to ordinary images
        # use https://docs.w3cub.com/tensorflow~2.3/keras/preprocessing/image/array_to_img
        #### Your Code goes here
        img = tf.keras.preprocessing.image.array_to_img(display_list[i])
        img_array = np.array(img)

        plt.imshow(img_array)
        plt.axis('off')
        ####
        plt.axis('off')
    plt.show()

Creating train and train_masks directories for stroring the processed images

In [115]:
!mkdir train
!mkdir train_masks

mkdir: cannot create directory ‘train’: File exists
mkdir: cannot create directory ‘train_masks’: File exists


In [116]:
data_dir = '/content/ss_dataset'
img_size = 256

image_root = '/content/train'
label_root = '/content/train_masks'

if not os.path.isdir(image_root):
    os.mkdir(image_root)
if not os.path.isdir(label_root):
    os.mkdir(label_root)

images = list()
labels = list()

"""
Beacause your images stored beside correspond label, and their format is in .bmp, so
please first store path of all images in a common list.
Then separate labels (which have _label pattern in their name) from images and store them in
images and labels lists.
For each image and label file, read it with PIL image lib. and resize them to 256*256.
Then convert them to png file
At last save them in image_root and label_root directories in a way none of the files with the same name are lost
for reading images:
    https://www.geeksforgeeks.org/python-pil-image-open-method/
for saving images:
    https://www.geeksforgeeks.org/python-pil-image-save-method/
"""
################################################
############## YOUR CODES GO HERE ##############
################################################

for root, _, files in os.walk(data_dir):
  for _file in files:
    path = os.path.join(root, _file)
    if path.endswith('.bmp') and '_label' not in path:
      images.append(path)
    elif path.endswith('.bmp') and '_label' in path:
      labels.append(path)

for img_path, label_path in zip(images, labels):
  img = Image.open(img_path)
  img = img.resize((img_size, img_size))
  img_new_path = os.path.basename(img_path).split('.')[0] + '.png'
  img.save(os.path.join(image_root, img_new_path))

  label = Image.open(label_path)
  label = label.resize((img_size, img_size))
  label_new_path = os.path.basename(label_path).split('.')[0] + '.png'
  label.save(os.path.join(label_root, label_new_path))

In [117]:
print("Train set:  ", len(os.listdir("/content/train")))
print("Train masks:", len(os.listdir("/content/train_masks")))

Train set:   859
Train masks: 859


In [166]:
## Call the dataframe_creation function with apprieciate arguments to get a
## suitable dataframes for images and masks. Call the images pathes column's name "image_path"
## and column for masks name "mask_name"
## In the end, create a new column in df of images called "mask_path" and fill it with "mask_path" column of the mask's df
################################################
############## YOUR CODES GO HERE ##############
################################################
img_df = dataframe_creation(image_root, "image_path").sort_values(by='id')
lbl_df = dataframe_creation(label_root, "mask_name").sort_values(by='id')
lbl_df.set_index('id', inplace = True)
img_df.set_index('id', inplace = True)

img_df["mask_path"] = lbl_df["mask_name"]

print(img_df)

                 image_path                           mask_path
id                                                             
0      /content/train/0.png    /content/train_masks/0_label.png
1      /content/train/1.png    /content/train_masks/1_label.png
10    /content/train/10.png   /content/train_masks/10_label.png
100  /content/train/100.png  /content/train_masks/100_label.png
101  /content/train/101.png  /content/train_masks/101_label.png
..                      ...                                 ...
95    /content/train/95.png   /content/train_masks/95_label.png
96    /content/train/96.png   /content/train_masks/96_label.png
97    /content/train/97.png   /content/train_masks/97_label.png
98    /content/train/98.png   /content/train_masks/98_label.png
99    /content/train/99.png   /content/train_masks/99_label.png

[859 rows x 2 columns]


In [174]:

img_size = [256, 256]

def data_augmentation(img, mask_img):
    """
    A function for data augmentation.
    We wanna just do some flips.
    Just make a random number, if it was greater than 0.5 do a lef_right flip
    hint:
      https://www.tensorflow.org/api_docs/python/tf/random/uniform
      https://www.tensorflow.org/api_docs/python/tf/image/flip_left_right
    Arguments:
      img: image tensor
      mask_img: mask image tensor
    return:
      img, mask_img
    """
    ################################################
    ############## YOUR CODES GO HERE ##############
    rand_num = tf.random.uniform(shape=(), minval=0, maxval=1)

    if rand_num > 0.5 :
      img = tf.image.flip_left_right(img)
      mask_img = tf.image.flip_left_right(mask_img)
    return img, mask_img
    ################################################

def preprocessing(path, mask_path):
    '''
    Do the usual preprocessing steps for image processing algorithms
    Read image tensors. decode them, resize to img_size, cast them fo float dtype and normalize between 0-1
    Hint:
      https://www.tensorflow.org/api_docs/python/tf/io/read_file
      https://www.tensorflow.org/api_docs/python/tf/io/decode_jpeg
      https://www.tensorflow.org/api_docs/python/tf/image/resize
      https://www.tensorflow.org/api_docs/python/tf/cast
    Set channels in decoding to 3
    Arguments:
      path: image path
      mask_path: mask image path
    return:
      pre_processed image and mask image tensors
    '''
    ################################################
    ############## YOUR CODES GO HERE ##############
    img = tf.io.read_file(path)
    mask_img = tf.io.read_file(mask_path)

    img = tf.image.decode_jpeg(img, channels=3)
    mask_img = tf.image.decode_jpeg(mask_img, channels=3)

    img = tf.image.resize(img, img_size)
    mask_img = tf.image.resize(mask_img, img_size)

    img = tf.cast(img, dtype=tf.float32) / 255.0
    mask_img = tf.cast(mask_img, dtype=tf.float32) / 255.0

    return img, mask_img
    ################################################

def create_dataset(df, train = False):
    '''
    A function for applying preprocessing and augmentation steps.
    Augment data just in train mode.
    First make a Dataset of tensors to reach high speed and ability.
    Then apply needed steps.
    For creating dataset, please use tensorflow-tf-data-dataset-from_tensor_slices to get
      a dataset of images and correspondig masks path. use values of image_path and mask_path columns of your dataframe
    Then use map function of created ds and call above functions respectively.
    use tf.data.AUTOTUNE in map function
    Hint:
      https://www.geeksforgeeks.org/tensorflow-tf-data-dataset-from_tensor_slices/
      https://www.tensorflow.org/api_docs/python/tf/data/Dataset
    Arguments:
      df: dataframe of images with masks path.
      train: boolean for switching between train and inference mode.
    return:
      dataset
    '''
    ################################################
    ############## YOUR CODES GO HERE ##############
    image_paths = df['image_path'].values
    mask_paths = df['mask_path'].values

    ds = tf.data.Dataset.from_tensor_slices((image_paths, mask_paths))
    ds = ds.map(preprocessing, num_parallel_calls=tf.data.AUTOTUNE)

    if train is True:
      ds = ds.map(data_augmentation, num_parallel_calls=tf.data.AUTOTUNE)

    return ds
    ################################################

In [175]:
### first split the dataframe to train and val (with train_test_split) then create dataset from each df
train_df, valid_df = train_test_split(img_df)
train = create_dataset(train_df, train=True)
valid = create_dataset(valid_df, train=False)

In [89]:
TRAIN_LENGTH = len(train_df)
BATCH_SIZE = 24
BUFFER_SIZE = 1000

In [176]:
## The last step for creating a tf.data.Dataset is to batch and shuffle them
## for this please cache train dataset, then shuffle in the amount of BUFFER_SIZE, then
## batch them in the amount of BATCH_SIZE and repeat this process. Finally prefetch the train_data with
## buffer_size=tf.data.AUTOTUNE (https://www.tensorflow.org/api_docs/python/tf/data/Dataset)
## Hint: name of needed functions are in the description, just call
## for valid_dataset, just batch in the amount of BATCH_SIZE
################################################
############## YOUR CODES GO HERE ##############
train_dataset = train.cache()
train_dataset = train.shuffle(BUFFER_SIZE)
train_dataset = train.batch(BATCH_SIZE)
train_dataset = train.repeat()
train_dataset = train.prefetch(buffer_size=tf.data.AUTOTUNE)
valid_dataset = valid.batch(BATCH_SIZE)
################################################

In [177]:
## let's see some of the images ans masks
for i in range(5):
    for image, mask in train.take(i):
        sample_image, sample_mask = image, mask
        display([sample_image, sample_mask])

Output hidden; open in https://colab.research.google.com to view.

Till now we created our dataset, let's implement the model and train it on created dataset

In this notebook we're going to implement [U-Net](https://arxiv.org/abs/1505.04597) model for semantic segmentation.
For its backbone, we'll use [MobileNetV2](https://arxiv.org/abs/1801.04381) architecture.

## U-Net model for segmentation

In [183]:
"""
First load the pre-trained weights of mobilenetv2. Because amount of our data is limited its better to use
transfer learning. Set the input_shape of the model to img_size, and please don't include the top of this model.
According to U-Net architecture, because we have skip connections in multiple steps of backbone, please
get the below layers's endpoint from backbone:
    block_1_expand_relu   # 64x64
    block_3_expand_relu   # 32x32
    block_6_expand_relu   # 16x16
    block_13_expand_relu  # 8x8
    block_16_project
Store them in a list.
Create your model with mobilenetv2 input layer and extracted output layers.
Freeze backbone and don't train it.
Hint:
  https://www.tensorflow.org/api_docs/python/tf/keras/applications/mobilenet_v2/MobileNetV2
  https://www.tensorflow.org/api_docs/python/tf/keras/Model
"""

################################################
############## YOUR CODES GO HERE ##############
################################################

base_model = tf.keras.applications.MobileNetV2(input_shape=(256, 256, 3), include_top=False, weights='imagenet')
backbone_layer = [
    base_model.get_layer('block_1_expand_relu').output,
    base_model.get_layer('block_3_expand_relu').output,
    base_model.get_layer('block_6_expand_relu').output,
    base_model.get_layer('block_13_expand_relu').output,
    base_model.get_layer('block_16_project').output
]

model = tf.keras.Model(inputs=base_model.input, outputs=backbone_layer)

for layer in base_model.layers:
    layer.trainable = False

In [184]:
def upsample(filters, size):
    '''
    in this function we have to define the decoder section of the u-net model.
    we call it upsampling process.
    for each down-sampling in the backbone, we have a up-sampling in decoder.
    we should implement decoder in a way to double the spatial resolution of the input tensor.
    bacause the inputs, are our features, we should learn how to double the spatial resolution instead of
    resizing methods for ignoring inserting fake features to our feature vectors.
    it's a crucial statement.
    so use a transpose conv2d function and double spatial resolution of the input tensor
    this layer should get the filters as output channel, size for filter size, 2 for stride.
    use same padding in it.
    define the initializer for kernel initializer and use False for use_bias.
    Use a sequential model for stacking layers.
    Finally add a batchnormalization layer at the end of the model. (use dafult args)
    Hint:
      https://www.tensorflow.org/api_docs/python/tf/keras/Sequential
      https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2DTranspose
      https://www.tensorflow.org/api_docs/python/tf/keras/layers/BatchNormalization
    Atguments:
      filters: number of input channels
      size: spatial dimention of filters
    return:
      decoder layers
    '''
    initializer = tf.random_normal_initializer(0., 0.02)

    ################################################
    ############## YOUR CODES GO HERE ##############
    result = ....


    ################################################

    return result

up_stack = [
    upsample(512, 3),  # 4x4 -> 8x8
    upsample(256, 3),  # 8x8 -> 16x16
    upsample(128, 3),  # 16x16 -> 32x32
    upsample(64, 3),   # 32x32 -> 64x64
]

In [ ]:
def unet_model(output_channels):
    '''
    Now we have to stack backbone with decoder.
    First define a Input layer to get the images (consider the img_size).
    Then give the result to the backbone and get the outputs of this encoder.
    you will get a list of outputs.
    store the last element of this list for input of the decoder.
    Reverse the outputs list except last element and keep them as skip connections.
    now iterate up_stack and skips lists simultaneously, give the previous last feature vector to the
    decoder part and get the output. concat that with current skip from backbone and name it as same as
    the last output to make it input of the next decoder layer.
    Finally, use a transpose conv2d layer to get the final coarse map.
    this layer should get the output_channels as output channel, 3 for filter size, 2 for stride, sigmoid for activation function.
    use same padding in it.
    give the last output of the decor to it and use as your model's main output.
    Hint:
      http://man.hubwiz.com/docset/TensorFlow.docset/Contents/Resources/Documents/api_docs/python/tf/keras/layers/Input.html
      https://www.tensorflow.org/api_docs/python/tf/keras/layers/Concatenate
      https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2DTranspose
      https://www.tensorflow.org/api_docs/python/tf/keras/Model
    '''
    ################################################
    ############## YOUR CODES GO HERE ##############

    return ...
    ################################################

In [ ]:
## let's define the loss function. dice loss function is a good choice for semantic segmenation task
## See https://dev.to/_aadidev/3-common-loss-functions-for-image-segmentation-545o
def dice_coef(y_true, y_pred, smooth=1):
    intersection = K.sum(y_true * y_pred, axis=[1,2,3])
    union = K.sum(y_true, axis=[1,2,3]) + K.sum(y_pred, axis=[1,2,3])
    return K.mean( (2. * intersection + smooth) / (union + smooth), axis=0)

def dice_loss(in_gt, in_pred):
    return 1-dice_coef(in_gt, in_pred)

## define model with  output channel. Name it model.
## then compile the model with adam optimizer, dice loss
## please introduce binary_accuracy and dice coef to the model to log them as a metric during training
## hint: https://keras.io/api/models/model_training_apis/

############################################
######### YOUR CODES GO HERE ###############
############################################

## plot the graph of the model with each layer's shape
## hint: https://www.tensorflow.org/api_docs/python/tf/keras/utils/plot_model


############################################
######### YOUR CODE GOES HERE ###############
############################################

In [ ]:
for images, masks in train_dataset.take(1):
    for img, mask in zip(images, masks):
        sample_image = img
        sample_mask = mask
        break

def visualize(display_list):
    plt.figure(figsize=(15, 15))
    title = ['Input Image', 'True Mask', 'Predicted Mask']
    for i in range(len(display_list)):
        plt.subplot(1, len(display_list), i+1)
        plt.title(title[i])
        plt.imshow(tf.keras.preprocessing.image.array_to_img(display_list[i]))
        plt.axis('off')
    plt.show()

def show_predictions(sample_image, sample_mask):
    pred_mask = model.predict(sample_image[tf.newaxis, ...])
    pred_mask = pred_mask.reshape(img_size[0],img_size[1],1)
    visualize([sample_image, sample_mask, pred_mask])


## let's see some the outputs of pre-trained model
show_predictions(sample_image, sample_mask)

In [ ]:
model.summary()

In [ ]:
### first define a early stop callback for preventing overfitting.
### set patience=4 and True the restore_best_weight
### Hint: https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/EarlyStopping

############################################
######### YOUR CODE GOES HERE ###############
############################################

# Let's observe how the model improves while it is training.
# To accomplish this task, a callback function is defined below.
class DisplayCallback(tf.keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs=None):
        if (epoch + 1) % 5 == 0:
            show_predictions(sample_image, sample_mask)

EPOCHS = 30

### Define the STEPS_PER_EPOCH and fit the model
### use train_dataset and EPOCHS and STEPS_PER_EPOCH for training process
### use valid_dataset for validation and introduce callbacks to the model
### save history

############################################
######### YOUR CODES GO HERE ###############
############################################

In [ ]:
### let's see some the new predictions
for i in range(5):
    for image, mask in valid.take(i):
        sample_image, sample_mask = image, mask
        show_predictions(sample_image, sample_mask)

In [ ]:
### save final checkpoints to the disk
### Use h5 extension
### Hint: https://www.tensorflow.org/guide/keras/save_and_serialize

############################################
######### YOUR CODES GO HERE ###############
############################################